# We will use a machine learning framwork named pytorch
## following this [tutorial](https://youtu.be/tHL5STNJKag?si=TgT5kABIFReWEcbD)
## I dont actually understand what any of this stuff does

### I later found out that you have to install pytorch like this for it to access your GPU
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [2]:
# imports based on the tutorial
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

print(torch.cuda.is_available()) # this checks weathers the GPU can be used

True


In [3]:
class ChestXrayDataset(Dataset):
  def __init__(self, data_dir, transform=None):
    self.data = ImageFolder(data_dir, transform=transform)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

  @property
  def classes(self):
    return self.data.classes

In [4]:
dataset = ChestXrayDataset('ChestXRayDataset')

In [5]:
len(dataset)

5840

0 healthy   1 pneumonia

In [6]:
image, label = dataset[1575]
print(label)
# image

1


In [7]:
transform = transforms.Compose([
    # no need to resize images
    transforms.Resize((250, 250)),
    transforms.ToTensor()
])

dataset = ChestXrayDataset('ChestXRayDataset', transform=transform)

In [8]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

## delete code under?

In [9]:
for images, labels in dataloader:
    pass

# the model

In [10]:
class ChestXrayModel(nn.Module):

    def __init__(self, num_classes=2): # only 2 classes (healthy or pneumonia)
        super(ChestXrayModel, self).__init__()
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])
        enet_out_size = 1280
        self.classifier = nn.Linear(enet_out_size, num_classes)
        pass

    def forward(self, x):
        x = self.features(x)
        output = self.classifier(x)
        return output

In [11]:
model = ChestXrayModel(num_classes=2)

In [12]:
model(images)

tensor([[-0.0268,  0.1410],
        [ 0.3956, -0.1125],
        [ 0.0680, -0.1564],
        [ 0.1798, -0.2894],
        [-0.1724,  0.1242],
        [ 0.1471, -0.1342],
        [ 0.0940, -0.1997],
        [ 0.1817,  0.0101],
        [-0.0708, -0.0891],
        [-0.1379, -0.0258],
        [ 0.1199,  0.0387],
        [-0.0056,  0.1070],
        [ 0.0790, -0.1018],
        [-0.0350,  0.0233],
        [ 0.0357, -0.2032],
        [-0.0216, -0.0201]], grad_fn=<AddmmBackward0>)

# splitting the data into 3 datasets

In [ ]:
# import splitfolders

# input_folder = "ChestXrayDataset"
# output_folder = "splitted_data"

# splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0.7, 0.15, 0.15))

Copying files: 0 files [00:00, ? files/s]

Copying files: 5840 files [00:11, 519.87 files/s]


# creating the training loop 

In [14]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
transform = transforms.Compose([
    transforms.Resize((250, 250)),
    transforms.ToTensor()
])


train_dataset = ChestXrayDataset('splitted_data/train', transform=transform)
val_dataset   = ChestXrayDataset('splitted_data/val',   transform=transform)
test_dataset  = ChestXrayDataset('splitted_data/test',  transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(val_dataset, batch_size=32, shuffle=False)

### I cant get it to use my GPU.

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
num_epochs = 5
train_losses, val_losses = [], []

model = ChestXrayModel(num_classes=2)
model.to(device)

#
#
# training
#
#
for epoch in range(num_epochs):
    model.train() # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)

#
#
# validation
#
#
    model.eval() # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc='Validation loop'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)

    val_loss = running_loss / len(val_loader.dataset)
    val_losses.append(val_loss)

Validation loop:   0%|          | 0/28 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/28 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/28 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/28 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/28 [00:00<?, ?it/s]

# TODO: Visualize losses